In [1]:
pip install -e .

Obtaining file:///E:/pharmacodynamics
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for PharmOps (pyproject.toml): started
  Building editable for PharmOps (pyproject.toml): finished with status 'done'
  Created wheel for PharmOps: filename=PharmOps-0.2.1.post0+dirty-0.editable-py3-none-any.whl size=3212 sha256=e2f24dcb31b1e586b72aff662998ea46c78f6f54f9c738e9691379fe317a2173
  Stored in directory: C:\Users\bartlelo\AppData\Local\Temp\2\pip-ephem-wheel-cache-qwfbp72x\wheels\c6\05\80\c89cb64c2e8665925d84b443324e9e62ea216185fc2

In [2]:
import pandas as pd
import numpy as np
import PharmOps
from importlib.metadata import version

In [3]:
# import raw data spreadsheet
rawData = pd.read_excel('sample data\\Binding Template for RAW transformations.xlsx')
rawData

,Unnamed: 0,1,2,3,4,5,6,7,8,9,10,11,12,Unnamed: 13,Unnamed: 14
0,A,300,174,145,195,222,277,349,341,259,313.0,250.0,307.0,NaN,copy-paste from raw txt to left
1,B,344,214,140,196,243,292,436,338,276,272.0,270.0,312.0,NaN,NaN
2,C,373,212,209,202,234,331,389,341,326,358.0,318.0,369.0,NaN,NaN
3,D,506,190,149,263,325,321,374,372,341,334.0,333.0,385.0,NaN,NaN
4,E,358,265,209,282,487,421,454,398,335,441.0,343.0,334.0,NaN,NaN
5,F,565,162,158,364,435,425,497,476,352,474.0,327.0,413.0,NaN,NaN
6,G,361,176,161,261,262,387,430,380,314,375.0,396.0,422.0,NaN,NaN
7,H,352,197,149,279,280,394,502,347,337,446.0,395.0,353.0,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,assay date,2024-01-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 8 rows, labeled A-H in unnamed first column. 12 columns of wells
rowLabel = rawData.loc[rawData.index[0:8], ['Unnamed: 0']]
rawValues = rawData.loc[rawData.index[0:8], range(1, 12)]
print(rawValues)
print(rowLabel)

     1    2    3    4    5    6    7    8    9     10     11
0  300  174  145  195  222  277  349  341  259  313.0  250.0
1  344  214  140  196  243  292  436  338  276  272.0  270.0
2  373  212  209  202  234  331  389  341  326  358.0  318.0
3  506  190  149  263  325  321  374  372  341  334.0  333.0
4  358  265  209  282  487  421  454  398  335  441.0  343.0
5  565  162  158  364  435  425  497  476  352  474.0  327.0
6  361  176  161  261  262  387  430  380  314  375.0  396.0
7  352  197  149  279  280  394  502  347  337  446.0  395.0
  Unnamed: 0
0          A
1          B
2          C
3          D
4          E
5          F
6          G
7          H


In [5]:
# data loading
testBind = PharmOps.WellData(rawData)
testBind.display()
allDR = testBind.make_all_drug_reports()
dr=testBind.make_drug_report(1)
for i in allDR:
    print(i.average)
    print(i.specific)
    print(i.pctTotal)
    print(i.drug)

     1    2    3    4    5    6    7    8    9     10     11     12
0  300  174  145  195  222  277  349  341  259  313.0  250.0  307.0
1  344  214  140  196  243  292  436  338  276  272.0  270.0  312.0
2  373  212  209  202  234  331  389  341  326  358.0  318.0  369.0
3  506  190  149  263  325  321  374  372  341  334.0  333.0  385.0
4  358  265  209  282  487  421  454  398  335  441.0  343.0  334.0
5  565  162  158  364  435  425  497  476  352  474.0  327.0  413.0
6  361  176  161  261  262  387  430  380  314  375.0  396.0  422.0
7  352  197  149  279  280  394  502  347  337  446.0  395.0  353.0
[]
['Drug1', 'Drug2', 'Drug3', 'Drug4']
     3    4    5    6    7    8    9     10     11
0  145  195  222  277  349  341  259  313.0  250.0
1  140  196  243  292  436  338  276  272.0  270.0
     1    2    3    4    5    6    7    8    9     10     11     12
0  300  174  145  195  222  277  349  341  259  313.0  250.0  307.0
1  344  214  140  196  243  292  436  338  276  272.0  270.

In [ ]:
# object serialization and saving
import pickle
import h5py

filePath = "sample data/testH5.h5"
serializedObj = pickle.dumps(testBind)
serializedArray = np.frombuffer(serializedObj, dtype='uint8')
groupName = "testObjects"
datasetName = "testData"
with h5py.File(filePath, "a") as h5file:
    group = h5file.require_group(groupName)

    if datasetName in group:
        del group[datasetName]
        print(f"Dataset '{datasetName}' exists. Overwriting.")
    group.create_dataset("testData", data=serializedArray)
    metadata = h5file.require_group("metadata")
    currentVersion = version("PharmOps")
    metadata.attrs["version"] = currentVersion

SyntaxError: invalid syntax (2912385473.py, line 13)

In [ ]:
# object loading and deserialization
with h5py.File(filePath, "r") as h5file:
    if groupName not in h5file or datasetName not in h5file[groupName]:
        raise KeyError(f"Dataset '{datasetName}' not found in group '{groupName}'")
    
    serializedArray = h5file[groupName][datasetName][:]
    loadedRawData = pickle.loads(serializedArray.tobytes())
    print(list(h5file.keys()))

loadedRawData.display()

In [ ]:
# GUI sandbox
import tkinter as tk
from tkinter import ttk

root = tk.Tk()
root.title("binding GUI test")
root.geometry("625x300")

mainframe = ttk.Frame(root)
mainframe['borderwidth'] = 10
mainframe['relief'] = 'sunken'
mainframe.grid(row=0, column=0, sticky="N, S, E, W", padx=5, pady=5)
root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)


currentObj = loadedRawData
tree = ttk.Treeview(mainframe, columns=list(currentObj.data.columns), show="headings")
tree.grid(row=0, column=0, sticky="N, S, E, W")

for col in currentObj.data.columns:
    tree.heading(col, text=col)
    tree.column(col, width=100, anchor=tk.CENTER)

for _, row in currentObj.data.iterrows():
    tree.insert("", tk.END, values=list(row))

mainframe.grid_rowconfigure(0, weight=1)
mainframe.grid_columnconfigure(0, weight=1)

scroll_x = ttk.Scrollbar(mainframe, orient=tk.HORIZONTAL, command=tree.xview)
scroll_x.grid(row=1, column=0, sticky="E, W")
scroll_y = ttk.Scrollbar(mainframe, orient=tk.VERTICAL, command=tree.yview)
scroll_y.grid(row=0, column=1, sticky="N, S")

tree.configure(xscrollcommand=scroll_x.set, yscrollcommand=scroll_y.set)

label = ttk.Label(root, text="more text")
label.grid(row=1, column=0, sticky="W", padx=5, pady=5)

button = ttk.Button(root, text="Click me")
button.grid(row=2, column=0, sticky="W", padx=5, pady=5)
root.mainloop()